In [25]:
import os
import sys
import glob
import gzip
import pandas
import pickle
from Bio import SeqIO

In [24]:
refseq_dir = '/ccb/salz4-4/markus/genome_data/refseq_singlestrain'
taxid_map_path = r'/ccb/salz4-4/markus/markraken/data/refseq_singlestrain/nucl_gb.accession2taxid'
txid_map_picklepath = r'/ccb/salz4-4/markus/markraken/data/refseq_singlestrain/refseq_accession2taxid.pkl'
fasta_combined_path = '/ccb/salz4-4/markus/markraken/data/refseq_singlestrain/refseq.fa'

In [3]:
refseq_to_taxid_df = pandas.read_csv(taxid_map_path, delimiter='\t')

In [9]:
# refseq_accession_list = refseq_to_taxid_df['accession'].tolist())
foo = refseq_to_taxid_df.loc[refseq_to_taxid_df['accession.version'] == 'NZ_CP032382.1', 'taxid'].iloc[0]

In [21]:
refseq_to_taxid = dict(zip(refseq_to_taxid_df['accession.version'],refseq_to_taxid_df['taxid']))

In [28]:
with open(txid_map_picklepath, 'wb') as f:
    pickle.dump(refseq_to_taxid, f)

In [30]:
allfile = []
for filename in glob.iglob(refseq_dir + '**/**', recursive=True):
    allfile.append(filename)

In [37]:
fastafile = [x for x in allfile if x[-6:]=='fna.gz']

In [49]:
seq_list = []
taxid_list = []
n_pass = 0
n_fail = 0

for path in fastafile:
    with gzip.open(path, 'rt') as f:
        record_list = list(SeqIO.parse(f, "fasta"))
        for r in record_list:
            try:
                taxid = refseq_to_taxid[r.id]
                seq = r.seq
                taxid_list.append(taxid)
                seq_list.append(str(seq))
                n_pass += 1
            except:
                n_fail += 1
#         sys.exit()

In [48]:
print(seq_list[0][:100])
print(taxid_list)

GCCCTGACCGTGGCCGGCTCCTTTGCCGATTCAAAAAATGTATATGACATGATGGGGAAAGCACTTGAACCCATGGGGTTGAAGCTCGACTACGCCTACA
[2321403]


In [51]:
print(n_pass, n_fail)
print(len(seq_list))
print(len(taxid_list))

6039 15
6039
6039


In [66]:
# save fasta
n_base_per_line = 60
with open(fasta_combined_path, 'w') as f:
    for i in range(len(seq_list)):
        f.write('>foo|bar|' + str(taxid_list[i]) + '\n')
        s = seq_list[i]
        seqlines = [s[i:i+n] for i in range(0, len(s), n_base_per_line)]
        for l in seqlines:
            f.write(l + '\n')